In [163]:
import requests
import json
from Crypto.PublicKey import RSA
import time
import datetime

import pandas as pd

from bs4 import BeautifulSoup

import pyperclip

import getpass

from util.crypto import encrypt_data
from util.html_tools import get_variable_from_html, get_json_variable_from_html

In [10]:
def create_steam_auth_session():
    session = requests.Session()
    response = session.get("https://steamcommunity.com/login/home/?goto=")
    user = input("Steam Username:")
    password = getpass.getpass("Password:")
    login_params = {
        "username": user
    }

    response = session.post("https://steamcommunity.com/login/home/getrsakey/",
        params = login_params)
    responseJSON = json.loads(response.text)
    
    
    exp = int(responseJSON["publickey_exp"],16)
    mod = int(responseJSON["publickey_mod"],16)
    
    rsa_key = RSA.construct((mod, exp))
    
    encrypted_password = encrypt_data(password, rsa_key)
    
    twofactorcode = ""
    m_gidCaptcha = "-1"
    captchaText = ""
    m_steamidEmailAuth = ""
    m_unRequestedTokenType = "-1"
    login_params = {
        "donotcache":round(time.time() * 1000),
        "password":encrypted_password,
        "username":user,
        "twofactorcode": twofactorcode,
        "emailauth":"",
        "loginfriendlyname":"",
        "captchagid":m_gidCaptcha,
        "captcha_text":captchaText,
        "emailsteamid":m_steamidEmailAuth,
        "rsatimestamp":responseJSON["timestamp"],
        "remember_login": 'false',
        "tokentype": m_unRequestedTokenType
        
    }
    
    header = {
        "Content-Type":"application/x-www-form-urlencoded; charset=UTF-8",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
    }
    
    
    response = session.post("https://steamcommunity.com/login/home/dologin",
                     data = login_params
                     )
    responseJSON = json.loads(response.text)

    if responseJSON["success"] == True:
        print(f"Successfully logged in {user}")
        return s

    require_twofactor = False
    require_email = False
    if "requires_twofactor" in responseJSON:
        if responseJSON["requires_twofactor"] == True:
            require_twofactor = True
    if "emailauth_needed" in responseJSON:
        if responseJSON["emailauth_needed"] == True:
            require_email = True

    if not require_email and not require_twofactor:
        print("Session could not be established, wrong password or username")
        return None
    
    if require_twofactor:
        twofactorcode = input("Enter your 2FA code:")
        login_params["twofactorcode"] = twofactorcode
    else:
        email_code = input("Enter your email confirmation code:")
        login_params["emailauth"] = email_code

    response = session.post("https://steamcommunity.com/login/home/dologin",
                 data = login_params
                 )
    responseJSON = json.loads(response.text)
    if responseJSON["success"] == True:
        return session
    else:
        print("Login failed")
        print(responseJSON)
    
        

In [33]:
def create_inventory_history_list(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    history_list = []
    rows = soup.find_all("div", "tradehistoryrow")
    for row in rows:
        history_element = {
            "description": "",
            "timestamp": None,
            "new_items": [],
            "lost_items": [],
        }
        date = row.find("div", "tradehistory_date").text.strip().split("\t")
        date = [x for x in date if x]
        day = int(date[0].split(" ")[0])
        month = date[0].split(" ")[1][:-1]
        year = int(date[0].split(" ")[-1])
        hour = int(date[1].split(":")[0])
        minute = int(date[1].split(":")[1][:-2])
        ampm = date[1][-2:]
        history_element["timestamp"] = time.mktime(datetime.datetime.strptime(f"{day:02d}/{month}/{year} {hour:02d}:{minute:02d} {ampm}", "%d/%b/%Y %H:%M %p").timetuple())
        history_element["description"] = row.find("div", "tradehistory_event_description").text.strip()
        item_changegroups = row.find_all("div", "tradehistory_items tradehistory_items_withimages")
        #print(history_element)
        for item_changegroup in item_changegroups:
            item_type = item_changegroup.find("div", "tradehistory_items_plusminus").text.strip()
            items = item_changegroup.find_all("a", "history_item economy_item_hoverable")
            items += item_changegroup.find_all("span", "history_item economy_item_hoverable")
            item_list = []
            for item in items:
                item_dict = {}
                item_dict["data-classid"] = item["data-classid"]
                item_dict["data-instanceid"] = item["data-instanceid"]
                item_list.append(item_dict)
            if item_type == "+":
                history_element["new_items"] += item_list
            elif item_type == "-":
                history_element["lost_items"] += item_list
        
        history_list.append(history_element)
    return history_list

In [148]:
def get_inventory_history(session):
    print("Fetching Inventory History")
    response = session.get("https://steamcommunity.com/")
    if response.status_code != 200:
        return None
    steamid = get_variable_from_html("g_steamID", response.text)
    response = session.get(f"https://steamcommunity.com/profiles/{steamid}/inventoryhistory/?app[]=730")
    if response.status_code != 200:
        return None
    profile_link = get_variable_from_html("g_strProfileURL", response.text).replace("\\", "")
    sessionid = get_variable_from_html("g_sessionID", response.text)
    filter_apps = get_variable_from_html("g_rgFilterApps", response.text)


    cursor = get_variable_from_html("g_historyCursor", response.text)
    item_json = get_json_variable_from_html("g_rgDescriptions", response.text)

    full_site = response.text
    item_dict = item_json["730"]
    history = create_inventory_history_list(full_site)
    print()

    while True:
        print(f"\rCurrently registered {len(history)} inventory changes")
        req_data = {
            "ajax": "1",
            "sessionid": sessionid,
        }
        for key in cursor:
            req_data[f"cursor[{key}]"] = cursor[key]
        for num, app in enumerate(filter_apps):
            req_data[f"app[{num}]"] = app
        response = session.get(profile_link+"/inventoryhistory/", params=req_data)
        if response.status_code != 200:
            print("")
            for i in range(30, 0, -1):
                print(f"\rWaiting for {i} seconds due to rate limit")
                time.sleep(1)
            print("Continuing")
            continue
        response_JSON = json.loads(response.text)
        if "html" in response_JSON:
            history += create_inventory_history_list(response_JSON["html"])
        if "descriptions" in response_JSON:
            item_dict = {**item_dict, **response_JSON["descriptions"]["730"]}
        if "cursor" in response_JSON.keys():
            cursor = response_JSON["cursor"]
        else:
            break  
    return history, item_dict

In [116]:
session = create_steam_auth_session()

In [119]:
inventory_history, item_json = get_inventory_history(session)

Fetching Inventory History
Currently registered 50 inventory changes
Currently registered 100 inventory changes
Currently registered 150 inventory changes
Currently registered 200 inventory changes
Currently registered 250 inventory changes
Currently registered 300 inventory changes
Currently registered 350 inventory changes
Currently registered 400 inventory changes
Currently registered 450 inventory changes
Currently registered 500 inventory changes
Currently registered 550 inventory changes
Currently registered 600 inventory changes
Currently registered 650 inventory changes
Currently registered 700 inventory changes
Currently registered 750 inventory changes
Currently registered 800 inventory changes
Currently registered 850 inventory changes
Currently registered 900 inventory changes
Currently registered 950 inventory changes
Currently registered 1000 inventory changes
Currently registered 1050 inventory changes
Currently registered 1100 inventory changes
Currently registered 1150

In [151]:
def get_case_stats(inventory_history, item_json):
    df = pd.DataFrame(inventory_history)
    df["time"] = pd.to_datetime(df['timestamp'],unit='s')
    #df.drop(["timestamp"], inplace=True)
    case_openings = df[df["description"]=="Unlocked a container"]
    drops = df[df["description"]=="Got an item drop"]
    drops = drops.assign(item_name= drops["new_items"].map(lambda x: item_json[f'{x[0]["data-classid"]}_{x[0]["data-instanceid"]}']["market_name"]))
    drops = drops.assign(item_rarity = drops["new_items"].map(lambda x: next(item["name"] for item in item_json[f'{x[0]["data-classid"]}_{x[0]["data-instanceid"]}']["tags"] if item["category"] == "Rarity")))

    case_openings = case_openings.assign(new_item_name = case_openings["new_items"].map(lambda x: item_json[f'{x[0]["data-classid"]}_{x[0]["data-instanceid"]}']["market_name"]))
    case_openings = case_openings.assign(case_name = case_openings["lost_items"].map(lambda x: next(item for item in [ item_json[f'{it["data-classid"]}_{it["data-instanceid"]}']["market_name"] for it in x] if "Key" not in item)))
    case_openings = case_openings.assign(new_item_rarity = case_openings["new_items"].map(lambda x: next(item["name"] for item in item_json[f'{x[0]["data-classid"]}_{x[0]["data-instanceid"]}']["tags"] if item["category"] == "Rarity")))

    weapon_cases = case_openings[~case_openings["new_item_name"].str.contains("Sticker")]
    weapon_cases = weapon_cases[~weapon_cases["case_name"].str.contains("Pins")]
    weapon_cases = weapon_cases[~weapon_cases["case_name"].str.contains("Graffiti")]
    weapon_cases = weapon_cases[~weapon_cases["case_name"].str.contains("Patch")]
    weapon_cases = weapon_cases[~weapon_cases["case_name"].str.contains("Souvenir")]

    return weapon_cases, drops

    

In [164]:
def print_case_stats(cases):
    ret_string = ""
    ret_string += f"Total amount of cases opened: {len(cases.index)}\n"
    ret_string += "\n"
    #for case_type in cases["case_name"].unique():
    #    ret_string += f'{case_type}: {len(cases[cases["case_name"]==case_type])}\n'

    ret_string += (f'Knives: {len(weapon_cases[weapon_cases["new_item_name"].str.contains("★")])}\n')
    weapon_cases_noknife = weapon_cases[~weapon_cases["new_item_name"].str.contains("★")]
    ret_string += (f'Covert: {len(weapon_cases_noknife[weapon_cases_noknife["new_item_rarity"].str.contains("Covert")])}\n')
    ret_string += (f'Classified: {len(weapon_cases[weapon_cases["new_item_rarity"].str.contains("Classified")])}\n')
    ret_string += (f'Restricted: {len(weapon_cases[weapon_cases["new_item_rarity"].str.contains("Restricted")])}\n')
    ret_string += (f'Mil-Spec Grade: {len(weapon_cases[weapon_cases["new_item_rarity"].str.contains("Mil-Spec Grade")])}\n')

    pyperclip.copy(ret_string)

    print(ret_string)

In [ ]:
Classified         11
Covert              5
Mil-Spec Grade    304
Restricted         45

In [152]:
cases, drops = get_case_stats(htm_site, item_json)

In [165]:
print_case_stats(cases)

Total amount of cases opened: 365

Knives: 3
Covert: 2
Classified: 11
Restricted: 45
Mil-Spec Grade: 304



In [120]:
df = pd.DataFrame(htm_site)
df["time"] = pd.to_datetime(df['timestamp'],unit='s')
case_openings = df[df["description"]=="Unlocked a container"]
drops = df[df["description"]=="Got an item drop"]

In [121]:
drops = drops.assign(item_name= drops["new_items"].map(lambda x: item_json[f'{x[0]["data-classid"]}_{x[0]["data-instanceid"]}']["market_name"]))
drops = drops.assign(item_rarity = drops["new_items"].map(lambda x: next(item["name"] for item in item_json[f'{x[0]["data-classid"]}_{x[0]["data-instanceid"]}']["tags"] if item["category"] == "Rarity")))

In [122]:
case_openings = case_openings.assign(new_item_name = case_openings["new_items"].map(lambda x: item_json[f'{x[0]["data-classid"]}_{x[0]["data-instanceid"]}']["market_name"]))
case_openings = case_openings.assign(case_name = case_openings["lost_items"].map(lambda x: next(item for item in [ item_json[f'{it["data-classid"]}_{it["data-instanceid"]}']["market_name"] for it in x] if "Key" not in item)))
case_openings = case_openings.assign(new_item_rarity = case_openings["new_items"].map(lambda x: next(item["name"] for item in item_json[f'{x[0]["data-classid"]}_{x[0]["data-instanceid"]}']["tags"] if item["category"] == "Rarity")))
case_openings.sort_values(by=["case_name"])

,description,timestamp,new_items,lost_items,time,new_item_name,case_name,new_item_rarity
949,Unlocked a container,1.617932e+09,"[{'data-classid': '4302203205', 'data-instance...","[{'data-classid': '4292377293', 'data-instance...",2021-04-09 01:27:00,Sticker | Fnatic | 2020 RMR,2020 RMR Challengers,High Grade
964,Unlocked a container,1.617846e+09,"[{'data-classid': '4302203164', 'data-instance...","[{'data-classid': '4292377293', 'data-instance...",2021-04-08 01:40:00,Sticker | GODSENT | 2020 RMR,2020 RMR Challengers,High Grade
963,Unlocked a container,1.617846e+09,"[{'data-classid': '4302203121', 'data-instance...","[{'data-classid': '4292377293', 'data-instance...",2021-04-08 01:40:00,Sticker | Astralis | 2020 RMR,2020 RMR Challengers,High Grade
943,Unlocked a container,1.617932e+09,"[{'data-classid': '4302203157', 'data-instance...","[{'data-classid': '4292377293', 'data-instance...",2021-04-09 01:28:00,Sticker | Fnatic (Holo) | 2020 RMR,2020 RMR Challengers,Remarkable
962,Unlocked a container,1.617846e+09,"[{'data-classid': '4302203164', 'data-instance...","[{'data-classid': '4292377293', 'data-instance...",2021-04-08 01:40:00,Sticker | GODSENT | 2020 RMR,2020 RMR Challengers,High Grade
...,...,...,...,...,...,...,...,...
345,Unlocked a container,1.636107e+09,"[{'data-classid': '4633023480', 'data-instance...","[{'data-classid': '4632904834', 'data-instance...",2021-11-05 10:11:00,Souvenir PP-Bizon | Anolis (Field-Tested),Stockholm 2021 Mirage Souvenir Package,Consumer Grade
370,Unlocked a container,1.635325e+09,"[{'data-classid': '4622722188', 'data-instance...","[{'data-classid': '4622695271', 'data-instance...",2021-10-27 09:01:00,Souvenir Dual Berettas | Oil Change (Factory New),Stockholm 2021 Vertigo Souvenir Package,Consumer Grade
2888,Unlocked a container,1.485485e+09,"[{'data-classid': '1441630916', 'data-instance...","[{'data-classid': '1441632948', 'data-instance...",2017-01-27 02:38:00,Sticker | The Ninja,Team Roles Capsule,High Grade
89,Unlocked a container,1.644026e+09,"[{'data-classid': '4741262269', 'data-instance...","[{'data-classid': '4741262253', 'data-instance...",2022-02-05 01:56:00,Sticker | Flick Shotter,The Boardroom Sticker Capsule,High Grade


In [145]:
weapon_cases = case_openings[~case_openings["new_item_name"].str.contains("Sticker")]
weapon_cases = weapon_cases[~weapon_cases["case_name"].str.contains("Pins")]
weapon_cases = weapon_cases[~weapon_cases["case_name"].str.contains("Graffiti")]
weapon_cases = weapon_cases[~weapon_cases["case_name"].str.contains("Patch")]
weapon_cases = weapon_cases[~weapon_cases["case_name"].str.contains("Souvenir")]

In [154]:
weapon_cases[weapon_cases["new_item_name"].str.contains("★")]

,description,timestamp,new_items,lost_items,time,new_item_name,case_name,new_item_rarity
4006,Unlocked a container,1.437365e+09,"[{'data-classid': '721281532', 'data-instancei...","[{'data-classid': '926978479', 'data-instancei...",2015-07-20 04:02:00,★ Karambit | Doppler (Factory New),Chroma 2 Case,Covert
4363,Unlocked a container,1.433728e+09,"[{'data-classid': '1013342137', 'data-instance...","[{'data-classid': '991959905', 'data-instancei...",2015-06-08 01:47:00,★ Falchion Knife | Slaughter (Factory New),Falchion Case,Covert
4427,Unlocked a container,1.432770e+09,"[{'data-classid': '721065419', 'data-instancei...","[{'data-classid': '926978479', 'data-instancei...",2015-05-27 23:48:00,★ M9 Bayonet | Ultraviolet (Field-Tested),Chroma 2 Case,Covert


In [147]:
weapon_cases.groupby(by=["new_item_rarity"])["new_item_rarity"].count()

new_item_rarity
Classified         11
Covert              5
Mil-Spec Grade    304
Restricted         45
Name: new_item_rarity, dtype: int64

In [130]:
weapon_cases[weapon_cases["case_name"].str.contains("Souvenir")]

,description,timestamp,new_items,lost_items,time,new_item_name,case_name,new_item_rarity


In [92]:
drops["new_items"].map(lambda x: next(item["name"] for item in item_json[f'{x[0]["data-classid"]}_{x[0]["data-instanceid"]}']["tags"] if item["category"] == "Rarity"))

66          Base Grade
72          Base Grade
127         Base Grade
129         Base Grade
131         Base Grade
             ...      
1313    Consumer Grade
1336        Base Grade
1338        Base Grade
1344        Base Grade
1347        Base Grade
Name: new_items, Length: 207, dtype: object

In [80]:
drops["new_items"].map(lambda x: f"this is {x}")

66      this is [{'data-classid': '2727227113', 'data-...
72      this is [{'data-classid': '3106076656', 'data-...
127     this is [{'data-classid': '2727227113', 'data-...
129     this is [{'data-classid': '3106076656', 'data-...
131     this is [{'data-classid': '2727227113', 'data-...
                              ...                        
1313    this is [{'data-classid': '310776580', 'data-i...
1336    this is [{'data-classid': '638240019', 'data-i...
1338    this is [{'data-classid': '720268538', 'data-i...
1344    this is [{'data-classid': '638240019', 'data-i...
1347    this is [{'data-classid': '384801319', 'data-i...
Name: new_items, Length: 207, dtype: object

In [77]:
drops["item_name"] = [0]

,description,timestamp,new_items,lost_items,time
66,Got an item drop,1.642756e+09,"[{'data-classid': '2727227113', 'data-instance...",[],2022-01-21 09:12:00
72,Got an item drop,1.637125e+09,"[{'data-classid': '3106076656', 'data-instance...",[],2021-11-17 04:57:00
127,Got an item drop,1.617522e+09,"[{'data-classid': '2727227113', 'data-instance...",[],2021-04-04 07:43:00
129,Got an item drop,1.615260e+09,"[{'data-classid': '3106076656', 'data-instance...",[],2021-03-09 03:15:00
131,Got an item drop,1.612591e+09,"[{'data-classid': '2727227113', 'data-instance...",[],2021-02-06 05:55:00
...,...,...,...,...,...
1313,Got an item drop,1.421133e+09,"[{'data-classid': '310776580', 'data-instancei...",[],2015-01-13 07:06:00
1336,Got an item drop,1.420886e+09,"[{'data-classid': '638240019', 'data-instancei...",[],2015-01-10 10:31:00
1338,Got an item drop,1.420796e+09,"[{'data-classid': '720268538', 'data-instancei...",[],2015-01-09 09:33:00
1344,Got an item drop,1.420784e+09,"[{'data-classid': '638240019', 'data-instancei...",[],2015-01-09 06:19:00


In [50]:
test_item_id = 20
item_json["730"][f"{hist[test_item_id]['new_items'][0]['data-classid']}_{hist[test_item_id]['new_items'][0]['data-instanceid']}"]

{'icon_url': '-9a81dlWLwJ2UUGcVs_nsVtzdOEdtWwKGZZLQHTxDZ7I56KU0Zwwo4NUX4oFJZEHLbXH5ApeO4YmlhxYQknCRvCo04DEVlxkKgpotLO_JAlf0vL3ZjRQ5dKxq4yCkP_gfe7UlT0AvMAnibuUrYjx2QOwqUA6ZGD3LYSccAY-aQnSrFW4x7jt0cSi_MOeRceSpgs',
 'icon_url_large': '-9a81dlWLwJ2UUGcVs_nsVtzdOEdtWwKGZZLQHTxDZ7I56KU0Zwwo4NUX4oFJZEHLbXH5ApeO4YmlhxYQknCRvCo04DEVlxkKgpotLO_JAlf0vL3ZjRQ5dKxq4yCkP_gDLfQhGxUppB0ib-QoYmjiwbh8xc4Ym6ndoXDdQ86aFrZ-FG-yb_t05K6usvLnSB9-n51lUAssIw',
 'icon_drag_url': '',
 'name': 'PP-Bizon | Breaker Box',
 'market_hash_name': 'PP-Bizon | Breaker Box (Minimal Wear)',
 'market_name': 'PP-Bizon | Breaker Box (Minimal Wear)',
 'name_color': 'D2D2D2',
 'background_color': '',
 'type': 'Industrial Grade SMG',
 'tradable': 1,
 'marketable': 1,
 'commodity': 0,
 'market_tradable_restriction': '7',
 'descriptions': [{'type': 'html', 'value': 'Exterior: Minimal Wear'},
  {'type': 'html', 'value': ' '},
  {'type': 'html',
   'value': 'The Bizon SMG is low-damage, but offers a uniquely designed high-capacity drum

In [24]:
soup.find_all("script")

[<script type="text/javascript">
 	var __PrototypePreserve=[];
 	__PrototypePreserve[0] = Array.from;
 	__PrototypePreserve[1] = Function.prototype.bind;
 	__PrototypePreserve[2] = HTMLElement.prototype.scrollTo;
 </script>,
 <script src="https://community.akamai.steamstatic.com/public/javascript/prototype-1.7.js?v=.55t44gwuwgvw" type="text/javascript"></script>,
 <script type="text/javascript">
 	Array.from = __PrototypePreserve[0] || Array.from;
 	Function.prototype.bind = __PrototypePreserve[1] || Function.prototype.bind;
 	HTMLElement.prototype.scrollTo = __PrototypePreserve[2] || HTMLElement.prototype.scrollTo;
 </script>,
 <script type="text/javascript">VALVE_PUBLIC_PATH = "https:\/\/community.akamai.steamstatic.com\/public\/";</script>,
 <script src="https://community.akamai.steamstatic.com/public/javascript/scriptaculous/_combined.js?v=OeNIgrpEF8tL&amp;l=english&amp;load=effects,controls,slider,dragdrop" type="text/javascript"></script>,
 <script src="https://community.akamai.s